# 注意:
##针对原文件做了修改，删除了原理说明等文字，和插图，仅仅留下关键代码；
## 代码已经过colab验证,生成了所有的模型。

In [2]:
#colab中运行jupyter文件的步骤：
# 1.挂载云盘
from google.colab import drive
drive.mount('/content/gdrive')

# 2.安装需要的软件
!pip3 install transformers
!pip3 install pytorch-crf==0.7.2

import os
def get_root_dir():
    if os.path.exists('/content/gdrive/MyDrive/第二次意图识别_toColab/'):
        return '/content/gdrive/MyDrive/第二次意图识别_toColab/' #在Colab里
    else:
        return './' #在本地

# 3.调用系统命令，切换到对应工程路径，相当于cd，但是直接!cd是不行的
print("change to path:",get_root_dir())
os.chdir(get_root_dir())

# 4.再次确认路径
print('current path:')
!pwd
print('ls in current path:')
!ls

# 不借助print()实现多输出结果的打印
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


Mounted at /content/gdrive
     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 17.2 MB/s 
     |████████████████████████████████| 3.3 MB 17.4 MB/s 
     |████████████████████████████████| 596 kB 19.1 MB/s 
     |████████████████████████████████| 61 kB 569 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
change to path: /content/gdrive/MyDrive/第二次意图识别_toColab/
current path:
/content/gdrive/MyDrive/第二次意图识别_toColab
ls in current path:
 data			  textcnn
 dataSet.pkl		  textrcnn
 model-cnn.h5		  textrnn
 model-rcnn.h5		  textrnn+Attention
 model-rnn+Attention.h5   TextRNN+Attention.ipynb
 model-rnn.h5		  transformerEncoder
 parameter.pkl		  意图识别推理及评测.ipynb
 runs			 '第二次进行意图识别(文本分类)-最新版.ipynb'


In [3]:
# hava a look version
import math
import torch
import torch.nn as nn
# import tensorflow as tf
# torch.__version__,tf.__version__
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#torch.device('cpu')#
print(device,torch.__version__)
# print(device,torch.__version__,tf.__version__)
# import tensorflow as tf
# import tensorboard
# tf.__version__,tensorboard.__version__

cuda 1.10.0+cu111


In [ ]:
#!rm -r *.pkl textrnn textcnn

rm: cannot remove 'textrnn': No such file or directory
rm: cannot remove 'textcnn': No such file or directory


# 数据准备

In [4]:
import os
import time
import gensim
import random
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
from operator import itemgetter
from collections import defaultdict
#from sklearn.cross_validation import train_test_split #早已不用
from sklearn.model_selection import train_test_split

# 准备好模型的参数
parameter = {
    'min_count_word':1,
#     'char2ind':None,
#     'ind2char':None,
#     'ind2embeding':None,
#     'output_size': None,
    'epoch':20,
    'batch_size':100,
    'embedding_dim':300,
    'hidden_size':128,
    'num_layers':2, #堆叠LSTM的层数，默认值为1
    'dropout':0.5,
    'cuda':device,
    'lr':0.001,
    'num_unknow':0,
    'max_len':1000,
    'd_k':60, #k,q,v的隐层节点数
    'd_q':60,
    'd_v':60,
    'd_ff':1024, #ffn的隐层节点数
    'n_heads':5,
    'n_layers':2,
}

def build_dataSet(data,parameter):
    # 构建训练集（chars，labels），构建词汇表（char2ind，ind2char）
    chars = []
    labels = []
    vocab = defaultdict(int)
    vocab['<pad>'] = parameter['min_count_word']
    for text,label in tqdm(zip(data.text,data.label)):
        chars.append(text.split())
        labels.append(label)
        for char in chars[-1]:
            vocab[char] += 1
    vocab['<unk>'] = parameter['min_count_word']
    for char in vocab:
        if vocab[char] < parameter['min_count_word']:
            del vocab[char]
    char2ind,ind2char = dict(zip(vocab.keys(),range(len(vocab)))), \
    dict(zip(range(len(vocab)),vocab.keys()))
    ind2embeding = np.random.randn(len(vocab), parameter['embedding_dim']).astype(np.float32) / np.sqrt(len(vocab))
    # 加载词向量
    w2v = gensim.models.Word2Vec.load('data/wiki/wiki.Mode')
    for ind,i in enumerate(char2ind.keys()):
        try:
            embedding = np.asarray(w2v.wv[i], dtype='float32')
            ind2embeding[ind] = embedding
        except:
            parameter['num_unknow'] += 1
    parameter['ind2char'] = ind2char
    parameter['char2ind'] = char2ind
    parameter['ind2embeding'] = ind2embeding
    parameter['output_size'] = len(set(labels))
    return np.array(chars),np.array(labels)

def batch_yield(chars,labels,parameter,shuffle = True):
    for train_epoch in range(parameter['epoch']):
        if shuffle:
            permutation = np.random.permutation(len(chars))
            chars = chars[permutation]
            labels = labels[permutation]
        max_len = 0
        batch_x,batch_y,len_x = [],[],[]
        for iters in tqdm(range(len(chars))):
            batch_ids = itemgetter(*chars[iters])(parameter['char2ind'])
            try:
                batch_ids = list(batch_ids)
            except:
                batch_ids = [batch_ids,0]
            if len(batch_ids) > max_len:
                max_len = len(batch_ids)
            batch_x.append(batch_ids)
            batch_y.append(labels[iters])
            len_x.append(len(batch_ids))
            if len(batch_x) >= parameter['batch_size']:
                batch_x = [np.array(list(itemgetter(*x_ids)(parameter['ind2embeding']))+[parameter['ind2embeding'][0]]*(max_len-len(x_ids))) for x_ids in batch_x]
                device = parameter['cuda']
                yield torch.from_numpy(np.array(batch_x)).to(device),torch.from_numpy(np.array(batch_y)).to(device).long(),len_x,True,None
                max_len,batch_x,batch_y,len_x = 0,[],[],[]
        batch_x = [np.array(list(itemgetter(*x_ids)(parameter['ind2embeding']))+[parameter['ind2embeding'][0]]*(max_len-len(x_ids))) for x_ids in batch_x]
        device = parameter['cuda']
        yield torch.from_numpy(np.array(batch_x)).to(device),torch.from_numpy(np.array(batch_y)).to(device).long(),len_x,True,train_epoch
        max_len,batch_x,batch_y,len_x = 0,[],[],[]
    yield None,None,None,False,None
    
def batch_yield_predict(chars,parameter):
    batch_x,batch_y = [],[]
    for iters in range(len(chars)):
        if chars[iters] in parameter['char2ind']:
            batch_x.append(parameter['ind2embeding'][parameter['char2ind'][chars[iters]]])
        else:
            batch_x.append(parameter['ind2embeding'][parameter['char2ind']['<unk>']])
    batch_x = [batch_x]
#     batch_y = [0]
    device = parameter['cuda']
    return torch.from_numpy(np.array(batch_x)).to(device)#,torch.from_numpy(np.array(batch_y)).to(device).long()

if os.path.exists('dataSet.pkl') and os.path.exists('parameter.pkl'):
    print('exist!!')
    [train_chars,test_chars,train_labels,test_labels] = pk.load(open('dataSet.pkl','rb'))
    parameter_copy = pk.load(open('parameter.pkl','rb'))
    for i in parameter_copy.keys():
        if i not in parameter:
            parameter[i] = parameter_copy[i]
        else:
            print(i,':',parameter[i])
    pk.dump(parameter,open('parameter.pkl','wb'))
    
else:
    print('not exist')
    data = pd.read_csv('data/classification_data/classification_data.csv')
    chars_src,labels_src = build_dataSet(data,parameter)
    # 划分训练集和测试集
    train_chars,test_chars,train_labels,test_labels = train_test_split(chars_src,labels_src, test_size=0.2, random_state=42)
    pk.dump([train_chars,test_chars,train_labels,test_labels],open('dataSet.pkl','wb'))
    pk.dump(parameter,open('parameter.pkl','wb'))

train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,x_len,keys,epoch = next(train_yield)
# sta = time.time()
# while 1:
#     a,b,c = next(train_yield)
#     if not c:
#         break
        
# print(time.time()-sta)
# train_chars
seqs,labels,x_len,_,_ = next(train_yield)
print('\n','*'*100)
seqs[:10].shape,labels[:10]

exist!!
min_count_word : 1
epoch : 20
batch_size : 100
embedding_dim : 300
hidden_size : 128
num_layers : 2
dropout : 0.5
cuda : cuda
lr : 0.001
num_unknow : 0
max_len : 1000
d_k : 60
d_q : 60
d_v : 60
d_ff : 1024
n_heads : 5
n_layers : 2


  0%|          | 100/306113 [00:09<7:56:27, 10.70it/s]


 ****************************************************************************************************


(torch.Size([10, 40, 300]),
 tensor([11,  8,  6, 11, 14, 12,  4,  6,  9, 10], device='cuda:0'))

In [ ]:
pkl_fd=open('parameter.pkl','rb')
data=pk.load(pkl_fd)
data.keys()

dict_keys(['min_count_word', 'epoch', 'batch_size', 'embedding_dim', 'hidden_size', 'num_layers', 'dropout', 'cuda', 'lr', 'num_unknow', 'max_len', 'd_k', 'd_q', 'd_v', 'd_ff', 'n_heads', 'n_layers', 'ind2char', 'char2ind', 'ind2embeding', 'output_size'])

In [ ]:
seqs.shape

torch.Size([100, 41, 300])

In [ ]:
seqs.shape,len(x_len),max(x_len)

(torch.Size([100, 41, 300]), 100, 41)

# textCNN

## textCNN模型实现

In [ ]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

# 构建分类模型
class TextCNN(nn.Module):
    def __init__(self, parameter):
        super(TextCNN, self).__init__()
        filter_size=(3,4,5)
        hidden_size = parameter['hidden_size']
        embedding_dim = parameter['embedding_dim']
        output_size = parameter['output_size']
        dropout = parameter['dropout']
        
        self.convs = nn.ModuleList([nn.Conv2d(1, hidden_size,(k, embedding_dim)) for k in filter_size])
        # ins:(batch:100,C:1,len(chars):40,embedding_dim:300)
        # covs:(3/4/5,300),kernel = 128
        # outs:(batch:100,C:128,40-3/4/5+1,300-300+1)->(100,128,38,1)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * len(filter_size), output_size)

    def forward(self, x):
        x = x.unsqueeze(1) # [batch, channel, word_num, embedding_dim] = [N,C,H,W] -> (-1, 1, 20, 300)
#         x = [F.relu(conv(x)).squeeze(3) for conv in self.convs] # len(filter_size) * (N, filter_num, H) -> 3 * (-1, 100, 18)
        x = [F.tanh(conv(x)).squeeze(3) for conv in self.convs] # len(filter_size) * (N, filter_num, H) -> 3 * (-1, 100, 18)
#         x = [F.sigmoid(conv(x)).squeeze(3) for conv in self.convs] # len(filter_size) * (N, filter_num, H) -> 3 * (-1, 100, 18)
#         x = [F.sigmoid(conv(x)).squeeze(3) for conv in self.convs] # len(filter_size) * (N, filter_num, H) -> 3 * (-1, 100, 18)
        out_new = []
#         print([i.shape for i in x],'x.shape')
        for output in x:
            try:
                out_new.append(F.max_pool1d(output,output.shape[2].item()).squeeze(2))
            except:
                out_new.append(F.max_pool1d(output,output.shape[2]).squeeze(2))
#         print([i.shape for i in out_new],'out_new.shape')
        x = out_new
        x = torch.cat(x, 1) # (N, filter_num * len(filter_size)) -> (163, 100 * 3)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [ ]:
# 查看网络结构
from torch.utils.tensorboard import SummaryWriter
x = torch.rand(100,32,300)
model = TextCNN(parameter)
 
with SummaryWriter(comment='textcnn') as w:
    w.add_graph(model, x)  

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


In [ ]:
model = TextCNN(parameter).to(parameter['cuda'])
train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,_,keys,epoch = next(train_yield)
seqs.shape,model(seqs).shape


  0%|          | 199/306113 [01:19<34:07:26,  2.49it/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


(torch.Size([100, 42, 300]), torch.Size([100, 15]))

## textCNN模型训练

In [ ]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 记录日志
shutil.rmtree('textcnn') if os.path.exists('textcnn') else 1
writer = SummaryWriter('./textcnn', comment='textcnn')

# 构建模型
model = TextCNN(parameter).to(parameter['cuda'])


# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.001, momentum=0.95, nesterov=True)
# optimizer = torch.optim.Adam(model.parameters(),lr = parameter['lr'], \
#                              weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()

# # 保存图
# train_yield = batch_yield(train_chars,train_labels,parameter)
# seqs,label,_,keys,epoch = next(train_yield)
# writer.add_graph(model, (seqs,))

# 准备迭代器
train_yield = batch_yield(train_chars,train_labels,parameter)

# 开始训练
loss_cal = []
min_loss = float('inf')
if os.path.exists('model-cnn.h5'):
  print('TextCNN model has been trainned')
else:
  with writer:
      while 1:
          seqs,label,_,keys,epoch = next(train_yield)
          if not keys:
              break
          out = model(seqs)
          loss = criterion(out, label)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loss_cal.append(loss.item())
          if epoch is not None:
              if (epoch+1)%1 == 0:
                  loss_cal = sum(loss_cal)/len(loss_cal)
                  if loss_cal < min_loss:
                      min_loss = loss_cal
                      torch.save(model.state_dict(), 'model-cnn.h5')
                  print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                        parameter['epoch'],loss_cal))
              writer.add_scalar('loss',loss_cal,global_step=epoch+1)
              loss_cal = [loss.item()]
      writer.flush()
      writer.close()
    
# torch.save(model.state_dict(), 'model-cnn.h5')

TextCNN(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 128, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 128, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=384, out_features=15, bias=True)
)

TextCNN model has been trainned


  0%|          | 99/306113 [00:02<2:06:58, 40.17it/s]


# textRNN

## textRNN模型实现

In [ ]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

# 构建分类模型
class TextRNN(nn.Module):
    def __init__(self, parameter):
        super(TextRNN, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        output_size = parameter['output_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size*2, output_size)
        
    def forward(self, x):
#         print(x.shape,'x.shape')
        out,(h, c)= self.lstm(x)
#         print(out.shape,'out.shape')
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
from torch.utils.tensorboard import SummaryWriter
x = torch.rand(100,32,300)
model = TextRNN(parameter)
 
with SummaryWriter(comment='textrnn') as w:
    w.add_graph(model, x)  

In [ ]:
model = TextRNN(parameter).to(parameter['cuda'])
train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,_,keys,epoch = next(train_yield)
seqs.shape,model(seqs).shape

  0%|          | 0/306113 [00:00<?, ?it/s]

(torch.Size([100, 40, 300]), torch.Size([100, 15]))

## textRNN模型训练

In [ ]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 记录日志
shutil.rmtree('textrnn') if os.path.exists('textrnn') else 1
writer = SummaryWriter('./textrnn', comment='textrnn')

# 构建模型
model = TextRNN(parameter).to(parameter['cuda'])

# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.1, momentum=0.95, nesterov=True)
# optimizer = torch.optim.Adam(model.parameters(),lr = parameter['lr'], \
#                              weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()

# 保存图
# train_yield = batch_yield(train_chars,train_labels,parameter)
# seqs,label,_,keys,epoch = next(train_yield)
# writer.add_graph(model, (seqs,))

# 准备迭代器
train_yield = batch_yield(train_chars,train_labels,parameter)

# 开始训练
loss_cal = []
min_loss = float('inf')
if os.path.exists('model-rnn.h5'):
  print('TextRNN model has been trainned')
else:
  with writer:
      while 1:
          seqs,label,_,keys,epoch = next(train_yield)
          if not keys:
              break
          out = model(seqs)
          loss = criterion(out, label)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loss_cal.append(loss.item())
          if epoch is not None:
              if (epoch+1)%1 == 0:
                  loss_cal = sum(loss_cal)/len(loss_cal)
                  if loss_cal < min_loss:
                      min_loss = loss_cal
                      torch.save(model.state_dict(), 'model-rnn.h5')
                  print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                        parameter['epoch'],loss_cal))
              writer.add_scalar('loss',loss_cal,global_step=epoch+1)
              loss_cal = [loss.item()]
      writer.flush()
      writer.close()
    
# torch.save(model.state_dict(), 'model-rnn.h5')

TextRNN(
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=15, bias=True)
)

TextRNN model has been trainned


# textRCNN

## textRCNN模型实现

In [ ]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

class TextRCNN(nn.Module):
    def __init__(self, parameter):
        super(TextRCNN, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        output_size = parameter['output_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        self.lstm = nn.LSTM(embedding_dim,hidden_size, \
                            num_layers, bidirectional=True, \
                            batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.fc_for_concat = nn.Linear(hidden_size * 2 + embedding_dim, hidden_size * 2)
    
    def forward(self, x):
        out,(h, c)= self.lstm(x)
#         print(out.shape,x.shape)
        
        out = self.fc_for_concat(torch.cat((x, out), 2))
#         print(out.shape)
        # 激活函数
        out = F.tanh(out)
#         print(out.shape)
        out = out.permute(0, 2, 1)
#         print(out.shape)
        try:
            out = F.max_pool1d(out, out.size(2).item())
        except:
            out = F.max_pool1d(out, out.size(2))
#         print(out.shape)
        out = out.squeeze(-1)
        out = self.dropout(out)
        out = self.fc(out)
        return out
    
# 查看形状
# model = TextRCNN(parameter).to(parameter['cuda'])
# a = model(seqs)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
x = torch.rand(100,32,300)
model = TextRCNN(parameter)
 
with SummaryWriter(comment='textrcnn') as w:
    w.add_graph(model, x)  

In [ ]:
# parameter['batch_size'] = 50
model = TextRCNN(parameter).to(parameter['cuda'])
train_yield = batch_yield(train_chars,train_labels,parameter)
seqs,label,_,keys,epoch = next(train_yield)
# print(seqs.shape)
_ = model(seqs)

  0%|          | 0/306113 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


## textRCNN模型训练

In [ ]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter
# 记录日志
shutil.rmtree('textrcnn') if os.path.exists('textrcnn') else 1
writer = SummaryWriter('./textrcnn', comment='textrcnn')

# 构建模型
model = TextRCNN(parameter).to(parameter['cuda'])

# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.001, momentum=0.95, nesterov=True)
# optimizer = torch.optim.Adam(model.parameters(),lr = parameter['lr'], \
#                              weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()

# # 保存图
# train_yield = batch_yield(train_chars,train_labels,parameter)
# seqs,label,_,keys,epoch = next(train_yield)
# writer.add_graph(model, (seqs,))

# 准备迭代器
train_yield = batch_yield(train_chars,train_labels,parameter)

# 开始训练
loss_cal = []
min_loss = float('inf')
if os.path.exists('model-rcnn.h5'):
  print('model-rcnn.h5 exist!')
else:
  print('model not exist!')
  with writer:
      while 1:
          seqs,label,_,keys,epoch = next(train_yield)
          if not keys:
              break
          out = model(seqs)
          loss = criterion(out, label)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loss_cal.append(loss.item())
          if epoch is not None:
              if (epoch+1)%1 == 0:
                  loss_cal = sum(loss_cal)/len(loss_cal)
                  if loss_cal < min_loss:
                      min_loss = loss_cal
                      torch.save(model.state_dict(), 'model-rcnn.h5')
                  print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                        parameter['epoch'],loss_cal))
              writer.add_scalar('loss',loss_cal,global_step=epoch+1)
              loss_cal = [loss.item()]
      writer.flush()
      writer.close()
      
  # torch.save(model.state_dict(), 'model-rcnn.h5')

1

TextRCNN(
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=15, bias=True)
  (fc_for_concat): Linear(in_features=556, out_features=256, bias=True)
)


  0%|          | 0/306113 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")

  0%|          | 99/306113 [00:04<3:42:49, 22.89it/s]

100%|██████████| 306113/306113 [00:58<00:00, 5207.22it/s]


epoch [1/20], Loss: 1.5645


100%|██████████| 306113/306113 [00:57<00:00, 5326.86it/s]


...

epoch [18/20], Loss: 0.7371


100%|██████████| 306113/306113 [00:56<00:00, 5417.73it/s]


epoch [19/20], Loss: 0.7294


100%|██████████| 306113/306113 [00:56<00:00, 5443.36it/s]

epoch [20/20], Loss: 0.7230


# textRNN+Attention（HAN）

## textRNN+Attention模型实现

In [ ]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

# 构建分类模型
class TextRNN_Attention(nn.Module):
    def __init__(self, parameter):
        super(TextRNN_Attention, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        output_size = parameter['output_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.fc_attention = nn.Linear(hidden_size*2, hidden_size*2)
        self.fc = nn.Linear(hidden_size*2, output_size)

        
        # attention
        self.w = nn.Parameter(torch.zeros(hidden_size * 2))
        
    def forward(self, x):
        out,(h, c)= self.lstm(x)
        alpha = F.softmax(torch.matmul(F.tanh(self.fc_attention(out)),self.w),dim = 1).unsqueeze(-1)
        print(alpha.shape)
#         alpha = F.softmax(torch.matmul(F.tanh(out),self.w),dim = 1).unsqueeze(-1)
#         print(out.shape,alpha.shape)
        out = F.relu(torch.sum(out * alpha,1))
#         print(out.shape)
        out = self.fc(out)
        return out

In [ ]:
rnn_attention = TextRNN_Attention(parameter)
rnn_attention(torch.rand(100,32,300)).shape

torch.Size([100, 32, 1])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


torch.Size([100, 15])

In [ ]:
from torch.utils.tensorboard import SummaryWriter
x = torch.rand(100,32,300)
model = TextRNN_Attention(parameter)
 
with SummaryWriter(comment='textrnn_attention') as w:
    w.add_graph(model, x)  

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


torch.Size([100, 32, 1])
torch.Size([100, 32, 1])
torch.Size([100, 32, 1])


## textRNN+Attention模型训练

In [ ]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 记录日志
shutil.rmtree('textrnn+Attention') if os.path.exists('textrnn+Attention') else 1
writer = SummaryWriter('./textrnn+Attention', comment='textrnn+Attention')

# 构建模型
model = TextRNN_Attention(parameter).to(parameter['cuda'])

# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.01, momentum=0.95, nesterov=True)
# optimizer = torch.optim.Adam(model.parameters(),lr = parameter['lr'], \
#                              weight_decay = 0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# 保存图
# train_yield = batch_yield(train_chars,train_labels,parameter)
# seqs,label,keys,epoch = next(train_yield)
# writer.add_graph(model, (seqs,))

# 准备迭代器
train_yield = batch_yield(train_chars,train_labels,parameter)

# 开始训练
loss_cal = []
min_loss = float('inf')
num = 0
if os.path.exists('model-rnn+Attention.h5'):
  print('model-rnn+Attention.h5 exist!')
else:
  with writer:
      while 1:
          num += 1
          seqs,label,_,keys,epoch = next(train_yield)
          if not keys:
              break
          out = model(seqs)
          loss = criterion(out, label)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          loss_cal.append(loss.item())
          if num % 100 == 0:
              print(loss.item())
          if epoch is not None:
              if (epoch+1)%1 == 0:
                  loss_cal = sum(loss_cal)/len(loss_cal)
                  if loss_cal < min_loss:
                      min_loss = loss_cal
                      torch.save(model.state_dict(), 'model-rnn+Attention.h5')
                  for param_group in optimizer.param_groups:
                      now_lr = param_group['lr']
                  print('epoch [{}/{}], Loss: {:.4f}, lr:{}'.format(epoch+1, \
                                                        parameter['epoch'],loss_cal,str(now_lr)))
                  scheduler.step()
              writer.add_scalar('loss',loss_cal,global_step=epoch+1)
              loss_cal = [loss.item()]
  #     writer.flush()
      writer.close()
      
  # torch.save(model.state_dict(), 'model-rnn.h5')

# transformerEncoder

## transformerEncoder模型实现

### 位置层

In [5]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

class PositionalEncoding(nn.Module):
    def __init__(self, parameter):#d_model, dropout=0.1, max_len=200):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=parameter['dropout'])
        d_model = parameter['embedding_dim']
        max_len = parameter['max_len']

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        print(pe,pe.shape)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
def get_attn_pad_mask(q_len_list, k_len_list):
    global device
    len_q = max(q_len_list)
    len_k = max(k_len_list)
    batch_size = len(q_len_list)
    pad_attn_mask =  torch.from_numpy(np.array([[False]*i+[True]*(len_k-i) for i in k_len_list])).unsqueeze(1)
    return pad_attn_mask.expand(batch_size, len_q, len_k).byte().to(device)  # [batch_size, len_q, len_k]
    

- Question：
    1. 位置层的作用？
        - 位置编码是Transformer框架中特有的组成部分，补充了Attention机制本身不能捕捉位置信息的缺陷
        
    2. 为什么用正余弦函数来处理位置层？
        - $论文中使用的Positional Encoding(PE)是正余弦函数，位置(pos)越小，波长越长，每一个位置对应的PE都是唯一的。同时作者也提到，之所以选用正余弦函数作为PE，是因为这可以使得模型学习到token之间的相对位置关系。因为对于任意一个偏移量K，PE_(pos+k)可以由PE_(pos)线性表示，差了几个\pi /2$
    
    3. 位置层如何使用？
        - $Positional Embedding的成分直接叠加于Embedding之上，使得每个token的位置信息和它的语义信息(embedding)充分融合，并被传递到后续所有经过复杂变换的序列表达中去。$

In [6]:
# have a test
test = PositionalEncoding(parameter).to(device)
mask = get_attn_pad_mask(x_len,x_len)
seqs.shape,test(seqs).shape,mask.shape,parameter['max_len'],parameter['embedding_dim']

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00]],

        [[ 8.4147e-01,  5.4030e-01,  8.0782e-01,  ...,  1.0000e+00,
           1.0633e-04,  1.0000e+00]],

        [[ 9.0930e-01, -4.1615e-01,  9.5231e-01,  ...,  1.0000e+00,
           2.1267e-04,  1.0000e+00]],

        ...,

        [[-8.9797e-01, -4.4006e-01,  9.8994e-01,  ...,  9.9365e-01,
           1.0582e-01,  9.9439e-01]],

        [[-8.5547e-01,  5.1785e-01,  6.9774e-01,  ...,  9.9364e-01,
           1.0592e-01,  9.9437e-01]],

        [[-2.6461e-02,  9.9965e-01, -1.6740e-01,  ...,  9.9363e-01,
           1.0603e-01,  9.9436e-01]]]) torch.Size([1000, 1, 300])


(torch.Size([100, 40, 300]),
 torch.Size([100, 40, 300]),
 torch.Size([100, 40, 40]),
 1000,
 300)

### block（multi-head self attention + Add & Normalize + Feed Forward Network）

*multi-head self attention + ADD & Normalize

$Attention通常可以进行如下描述，表示为将query(Q)和key-value pairs映射到输出上，其中query、每个key、每个value都是向量，输出是V中所有values的加权，其中权重是由Query和每个key计算出来的，计算方法分为三步：$

- $1.计算比较Q和K的相似度，用f来表示：$

$$f(Q,K_{i})_{i = 1,2,3}$$
- $2.将得到的相似度进行softmax归一化：$

$$\alpha_{i} = \frac{e^{f(Q,K_{i})}}{\sum_{j = 1}^{m}e^{f(Q,K_{i})}}_{j = 1,2,3}$$
- $3.针对计算出来的权重，对所有的values进行加权求和，得到Attention向量：$

$$\sum_{i=1}^{m}\alpha_{i}V_{i}$$
-  $ex1.相似度的计算有以下4种：$
    
$$点乘 dot product： f(Q,K_{i}) = Q^TK_{i}$$
$$权重 General： f(Q,K_{i}) = Q^TWK_{i}$$
$$拼接权重 Concat： f(Q,K_{i}) = W[Q^T;K_{i}]$$
$$感知器 Perceptron： f(Q,K_{i}) = V^Ttanh(WQ+UK_{i})$$

- $ex2.Q、K、V意义：$

$$其中Q、K、V是输入通过一层全连接得到，相当于W_{Q}^TX、W_{K}^TX、W_{V}^TX$$

- $整个过程相当于：$

$输入（768维的词向量序列）通过全连接，分别映射至Q、K、V（64*nheads维度上），通过Q^TK点乘得到Q和K之间的相似度，然后通过softmax得到Q在K上各个得分，Softmax(Q^TK)，这个softmax的分数决定了当前单词在每个句子中每个单词位置的表示程度；然后V^T(Q^TK)，这里实际上的意义在于保存对当前词的关注度不变的情况下，降低对不相关词的关注。$

In [7]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self,parameter):
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = parameter['d_k']

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
#         print(scores)
#         print('scores:',scores.shape)
        attn = nn.Softmax(dim=-1)(scores)
#         print(attn)
#         print('attn:',attn.shape)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
#         print('context:',context.shape)
        return context, attn
    
class MultiHeadAttention(nn.Module):
    def __init__(self,parameter):
        super(MultiHeadAttention, self).__init__()
        device = parameter['cuda']
        self.d_q,self.d_k,self.d_v,self.d_model,self.n_heads = parameter['d_q'],parameter['d_k'], \
        parameter['d_v'],parameter['embedding_dim'],parameter['n_heads']
        self.W_Q = nn.Linear(self.d_model, self.d_q * self.n_heads, bias=False)
        self.W_K = nn.Linear(self.d_model, self.d_k * self.n_heads, bias=False)
        self.W_V = nn.Linear(self.d_model, self.d_v * self.n_heads, bias=False)
        self.fc = nn.Linear(self.n_heads * self.d_v, self.d_model, bias=False)
        self.sdp = ScaledDotProductAttention(parameter).to(device)
        self.add_norm = nn.LayerNorm(self.d_model)
        
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
#         print('input-shape',input_Q.shape)
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, self.n_heads, self.d_q).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_q]
        K = self.W_K(input_K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]
#         print('QKV-shape',Q.shape,K.shape,V.shape)
#         print('test:',K.transpose(-1,-2).shape)
#         print('attn-shape0:',attn_mask.shape)
#         print(attn_mask.shape)
        attn_mask_new = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]
#         print('attn-shape1:',attn_mask.shape)
#         print(attn_mask.shape)
        context, attn = self.sdp(Q, K, V, attn_mask_new)
        context = context.transpose(1, 2).reshape(batch_size, -1, self.n_heads * self.d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        output = self.add_norm(output + residual)
        return output, attn

In [8]:
mha = MultiHeadAttention(parameter).cuda()
mask = get_attn_pad_mask(x_len,x_len)
out = mha(seqs,seqs,seqs,mask)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/cuda/Indexing.cu:963.)
  


- Question：
    1. Multi-Head Attention的作用？
        - 将模型分为多个头，期望其形成多个相互独立子空间，可以让模型去关注不同方面的信息。但是也有很多论文不这么认为。也有人认为是计算复杂度的取舍。同时multi-head相对来说也比较冗余，mask掉一定比例的head对结果影响不大
        
    2. LN、BN、GN作用和区别？
        - Layer Normalization是一个通用的技术，其本质是规范优化空间，加速收敛。
        - 。。。课后自己补充下哈

### Feed Forward Network

In [9]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self,parameter):
        self.d_ff,self.d_model = parameter['d_ff'],parameter['embedding_dim']
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(self.d_model, self.d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(self.d_ff, self.d_model, bias=False)
        )
        self.add_norm = nn.LayerNorm(self.d_model)
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return self.add_norm(output + residual) # [batch_size, seq_len, d_model]

- Question：
    1. Feed Forward Network的作用？
        - 其实，FFN的加入引入了非线性(ReLu激活函数)，变换了attention output的空间, 从而增加了模型的表现能力。把FFN去掉模型也是可以用的，但是效果差了很多。

### Encoder实现




In [10]:
class EncoderLayer(nn.Module):
    def __init__(self,parameter):
        super(EncoderLayer, self).__init__()
        device = parameter['cuda']
        self.enc_self_attn = MultiHeadAttention(parameter).to(device)
        self.pos_ffn = PoswiseFeedForwardNet(parameter).to(device)

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn

In [11]:
# have a test
test = EncoderLayer(parameter).to(device)
mask = get_attn_pad_mask(x_len,x_len)
context,attn = test(seqs,mask)
# attn
context.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/cuda/Indexing.cu:963.)
  


torch.Size([100, 40, 300])

In [12]:
class Encoder(nn.Module):
    def __init__(self,parameter):
        super(Encoder, self).__init__()
        n_layers = parameter['n_layers']
        self.pos_emb = PositionalEncoding(parameter)
        self.layers = nn.ModuleList([EncoderLayer(parameter) for _ in range(n_layers)])

    def forward(self, enc_inputs,len_inputs):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        '''
        enc_outputs = self.pos_emb(enc_inputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(len_inputs, len_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

In [13]:
class transformerEncoder(nn.Module):
    def __init__(self,parameter):
        super(transformerEncoder, self).__init__()
        output_dim = parameter['output_size']
        d_model = parameter['embedding_dim']
        device = parameter['cuda']
        self.encoder = Encoder(parameter).to(device)
        self.fc = nn.Linear(d_model, output_dim)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self,enc_inputs,len_inputs):
        enc_outputs, enc_self_attns = self.encoder(enc_inputs,len_inputs)
#         print(enc_outputs.shape)
#         enc_outputs = enc_outputs.permute(0, 2, 1)
        enc_outputs,_ = torch.max(enc_outputs, 1)
#         print(enc_outputs.shape)
        outputs = self.fc(enc_outputs)
        return outputs

In [14]:
test = transformerEncoder(parameter).to(device)
test(seqs,x_len).shape

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00]],

        [[ 8.4147e-01,  5.4030e-01,  8.0782e-01,  ...,  1.0000e+00,
           1.0633e-04,  1.0000e+00]],

        [[ 9.0930e-01, -4.1615e-01,  9.5231e-01,  ...,  1.0000e+00,
           2.1267e-04,  1.0000e+00]],

        ...,

        [[-8.9797e-01, -4.4006e-01,  9.8994e-01,  ...,  9.9365e-01,
           1.0582e-01,  9.9439e-01]],

        [[-8.5547e-01,  5.1785e-01,  6.9774e-01,  ...,  9.9364e-01,
           1.0592e-01,  9.9437e-01]],

        [[-2.6461e-02,  9.9965e-01, -1.6740e-01,  ...,  9.9363e-01,
           1.0603e-01,  9.9436e-01]]]) torch.Size([1000, 1, 300])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/cuda/Indexing.cu:963.)
  


torch.Size([100, 15])

In [15]:
seqs.shape,len(x_len)

(torch.Size([100, 40, 300]), 100)

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# x = torch.rand(100,32,300).to(device)
# x_len = torch.rand(100).to(device)

# model = transformerEncoder(parameter)
 
# with SummaryWriter(comment='textrnn_attention') as w:
#     w.add_graph(model, (x,x_len))  

## transformerEncoder模型训练

In [16]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 记录日志
shutil.rmtree('transformerEncoder') if os.path.exists('transformerEncoder') else 1
writer = SummaryWriter('./transformerEncoder', comment='transformerEncoder')

# 构建模型
model = transformerEncoder(parameter).to(parameter['cuda'])


# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.01, momentum=0.95, nesterov=True)
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
# optimizer = torch.optim.Adam(model.parameters(),lr = parameter['lr'], \
#                              weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()

# 保存图
# train_yield = batch_yield(train_chars,train_labels,parameter)
# seqs,label,x_len,keys,epoch = next(train_yield)
# writer.add_graph(model, (seqs,))

# 准备迭代器
train_yield = batch_yield(train_chars,train_labels,parameter)

# 开始训练
loss_cal = []
min_loss = float('inf')
num = 0 # 用于中间loss查看
if os.path.exists('model-transformerEncoder.h5'):
  print('model-transformerEncoder.h5 exist!')
else:
  with writer:
      while 1:
          seqs,label,x_len,keys,epoch = next(train_yield)
          if not keys:
              break
          num += 1
          out = model(seqs,x_len)
          loss = criterion(out, label)
          optimizer.zero_grad()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
          optimizer.step()
          loss_cal.append(loss.item())
          if num%100 == 0:
              print(loss.item())
          if epoch is not None:
              if (epoch+1)%1 == 0:
                  loss_cal = sum(loss_cal)/len(loss_cal)
                  if loss_cal < min_loss:
                      min_loss = loss_cal
                      torch.save(model.state_dict(), 'model-transformerEncoder.h5')
                  print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                        parameter['epoch'],loss_cal))
              writer.add_scalar('loss',loss_cal,global_step=epoch+1)
              loss_cal = [loss.item()]
  #     writer.flush()
      writer.close()
      
  # torch.save(model.state_dict(), 'model-cnn.h5')

Output hidden; open in https://colab.research.google.com to view.